In [126]:
import csv
import os
import random
from openai import OpenAI
import time
from tqdm import tqdm
import re

api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=api_key)
model = "gpt-4o-2024-08-06"; file_name = "LLM_measures_300_4o_julia.csv"
#model = "gpt-4o-mini"; file_name = "LLM_measures_300_mini_alien.csv"

In [127]:
def get_gpt4o_response(messages):
    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Error occurred: {e}. Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print(f"Failed after {max_retries} attempts. Error: {e}")
                return None

In [128]:
def parse_response_1(response):

    ''' Two numbers '''
    
    lines = response.split('\n')
    scores = []
    for line in lines:
        if line.strip().isdigit():
            scores.append(int(line))
    return scores

def parse_response_2(response):

    ''' Two numbers in a numbered list '''
    
    lines = response.split('\n')
    scores = []
    for line in lines:
        line = re.sub(r"^[^.]*\.\s*", "", line)
        if line.strip().isdigit():
            scores.append(int(line))
    return scores

In [129]:
def generate_survey(scenario):
    question1 = "Please rate the extent to which you agree with the following statement (0=completely disagree, 100=completely agree):\
        \n\nHow much do you agree: When it comes to thinking about how the severe injury could have been avoided, it is relevant to consider what Julia could have done differently."
    question2 = "Please rate the extent to which you agree with the following statement (0=completely disagree, 100=completely agree):\
        \n\nJulia could have done more to avoid the severe injury."

    # Prepare the survey text
    survey_text = f"Please read the following accident scenario: \n\n{scenario}\n\nPlease answer two questions about the scenario:\n\n 1. {question1}\n\n 2. {question2}."
    
    return survey_text

def generate_survey_2(scenario):
    question1 = "Please rate the extent to which you agree with the following statement (0=completely disagree, 100=completely agree):\
        \n\nHow much do you agree: When it comes to thinking about how the devastation to the city could have been avoided, it is relevant to consider what Mayor Richards could have done differently."
    question2 = "Please rate the extent to which you agree with the following statement (0=completely disagree, 100=completely agree):\
        \n\Mayor Richards could have done more to avoid the devastation to the city."

    # Prepare the survey text
    survey_text = f"Please read the following accident scenario: \n\n{scenario}\n\nPlease answer two questions about the scenario:\n\n 1. {question1}\n\n 2. {question2}."
    
    return survey_text



In [130]:
def main():
    results = []
    scenarios = [
        "Every day for the last three years, Julia took the same short walk around her neighborhood.\n\nToday for her daily walk, Julia decided to take a different walking path to avoid running into a neighbor who she owed a favor to.\n\nTragically, while she was walking on this new path, a dead tree branch suddenly and silently snapped above her, falling directly on her head and injuring her.",
        
        "Every day for the last three years, Julia took the same short walk around her neighborhood.\n\nToday for her daily walk, Julia decided to take a different walking path to avoid running into a neighbor who she owed a favor to.\n\nTragically, while she was walking on this new path, a dead tree branch suddenly and silently snapped above her, falling directly on her head and injuring her. All of this happened literally within a split second, and was not directly visible to Julia.",

        "Mayor Richards is the mayor of a large city in the United States.\n\nMayor Richards is a single-issue politician, intensely focused on catastrophe planning. He is well-known for establishing the city’s catastrophe planning department which controversially accounts for 90% of his city’s budget and whose sole function is to predict and protect the city and its people from large-scale disasters.\n\nLast week, a sudden and powerful alien invasion from outer space struck Mayor Richard’s city, demolishing the city’s downtown and emergency infrastructure, leaving millions homeless and severely injured, and hundreds of thousands missing. Mayor Richards’ catastrophe-planning department were entirely unprepared.\n\nBefore this catastrophe, there existed no verified reports of alien life.",

        "Every day for the last three years, Julia took the same short walk around her neighborhood.\n\nToday for her daily walk, Julia decided to take a different walking path.\n\nJulia ended up walking on a path parallel to the one she normally takes. Tragically, while Julia was walking on a new street, a dead tree branch suddenly snapped and fell on top of her, severely injuring Julia."

    ]
    
    for i in tqdm(range(300), desc="Processing surveys"):
        
        # Randomly select one of the two scenarios
        scenario_num = random.randint(0, 1)
        #scenario_num = 1
        
        scenario = scenarios[scenario_num]
        survey_text = generate_survey(scenario)
            
        messages = [        
            {"role": "system", "content": """You are a human participant in a survey.
                Respond naturally and honestly, mimicking human psychology as realistically as possible.
                Only provide a rating score without explanation or labels. Please do not use any bulleted or numbered lists.
                Note: Before you provide the rating, reflect on whether it resembles human behavior. If it doesn't, adjust it to better mimic a human-like response."""},                
            {"role": "user", "content": survey_text}
        ]

        while True:
        
            response = get_gpt4o_response(messages)
            
            if response:
                    
                scores = parse_response_1(response)
                if len(scores) == 2:
                    print(scores)
                    results.append([i+1, scenario, scenario_num] + scores)
                    break
                    
                scores = parse_response_2(response)
                if len(scores) == 2:
                    print(scores)
                    results.append([i+1, scenario, scenario_num] + scores)
                    break

    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['participant', 'scenario_text', 'scenario', 'counterfactual_rating', 'avoid_rating'])
        writer.writerows(results)

In [131]:
if __name__ == "__main__":
    main()

Processing surveys:   0%|                     | 1/300 [00:00<03:37,  1.37it/s]

[30, 20]


Processing surveys:   1%|▏                    | 2/300 [00:01<02:55,  1.70it/s]

[30, 20]


Processing surveys:   1%|▏                    | 3/300 [00:01<03:17,  1.50it/s]

...

[40, 20]


Processing surveys: 100%|██████████████████▉| 299/300 [03:18<00:00,  1.48it/s]

[60, 40]


Processing surveys: 100%|███████████████████| 300/300 [03:18<00:00,  1.51it/s]

[30, 20]
